In [1]:
%pip install  langchain openai pymysql google-search-results --upgrade -q

Note: you may need to restart the kernel to use updated packages.


In [3]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI

In [4]:
llm = ChatOpenAI(temperature=0)
tools = load_tools(["serpapi", "llm-math"], llm=llm)

In [6]:
from tabnanny import verbose

agent = initialize_agent(tools,
                         llm,
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                         verbose=True)
agent.run("OpenAI的老板是谁，他的年龄的0.5次方是多少？")



> Entering new  chain...
I need to find out who the boss of OpenAI is and then calculate their age raised to the power of 0.5.
Action: Search
Action Input: "OpenAI boss"
Observation: Sam Altman
Thought:Now I need to find out Sam Altman's age and calculate it raised to the power of 0.5.
Action: Search
Action Input: "Sam Altman age"
Observation: 38 years
Thought:Now I can calculate 38 raised to the power of 0.5.
Action: Calculator
Action Input: 38^0.5
Observation: Answer: 6.164414002968976
Thought:I now know the final answer.
Final Answer: The boss of OpenAI is Sam Altman and his age raised to the power of 0.5 is approximately 6.164414002968976.

> Finished chain.


'The boss of OpenAI is Sam Altman and his age raised to the power of 0.5 is approximately 6.164414002968976.'

# SQL Database Agent


## 连接数据库


In [50]:
from langchain.sql_database import SQLDatabase
db_user = "root"
db_password = "04m27d"
db_host = "localhost"
db_name = "sale"
db = SQLDatabase.from_uri(
    f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}")

## 设置 LLM、工具包和代理执行器


In [53]:
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents import create_sql_agent
from langchain.agents.agent_types import AgentType

# llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
llm = ChatOpenAI(model_name="gpt-3.5-turbo-16k", temperature=0)

toolkit = SQLDatabaseToolkit(db=db, llm=llm)
agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

## 使用自然语言查询数据库：

1. 描述一个表及其关系：


In [32]:
agent_executor.run("描述与销售记录相关的表及其关系")



> Entering new  chain...
Action: sql_db_list_tables
Action Input: ""
Observation: Customer, Product, Region, SalesData, Salesperson
Thought:I can query the Customer, Product, Region, SalesData, and Salesperson tables.
Now I should query the schema of these tables to see their relationships.
Action: sql_db_schema
Action Input: "Customer, Product, Region, SalesData, Salesperson"
Observation: 
CREATE TABLE `Customer` (
	id INTEGER NOT NULL AUTO_INCREMENT, 
	name VARCHAR(255) COLLATE utf8mb4_unicode_ci, 
	region_id INTEGER, 
	PRIMARY KEY (id), 
	CONSTRAINT customer_ibfk_1 FOREIGN KEY(region_id) REFERENCES `Region` (id)
)ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_unicode_ci

/*
3 rows from Customer table:
id	name	region_id
1	彩虹传媒有限公司	1
2	兰金电子网络有限公司	1
3	飞海科技信息有限公司	1
*/


CREATE TABLE `Product` (
	id INTEGER NOT NULL AUTO_INCREMENT, 
	name VARCHAR(100) COLLATE utf8mb4_unicode_ci, 
	price DECIMAL(10, 2), 
	PRIMARY KEY (id)
)ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_uni

'The tables related to sales records and their relationships are Customer, Product, Region, SalesData, and Salesperson.'

In [33]:
agent_executor.run("描述`SalesData`表")



> Entering new  chain...
Action: sql_db_list_tables
Action Input: ""
Observation: Customer, Product, Region, SalesData, Salesperson
Thought:The relevant table is SalesData. I should query the schema of this table.
Action: sql_db_schema
Action Input: "SalesData"
Observation: 
CREATE TABLE `SalesData` (
	id INTEGER NOT NULL AUTO_INCREMENT, 
	salesperson_id INTEGER, 
	product_id INTEGER, 
	customer_id INTEGER, 
	region_id INTEGER, 
	amount INTEGER, 
	total DECIMAL(10, 2), 
	sales_date DATE, 
	PRIMARY KEY (id), 
	CONSTRAINT salesdata_ibfk_1 FOREIGN KEY(salesperson_id) REFERENCES `Salesperson` (id), 
	CONSTRAINT salesdata_ibfk_2 FOREIGN KEY(product_id) REFERENCES `Product` (id), 
	CONSTRAINT salesdata_ibfk_3 FOREIGN KEY(customer_id) REFERENCES `Customer` (id), 
	CONSTRAINT salesdata_ibfk_4 FOREIGN KEY(region_id) REFERENCES `Region` (id)
)ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_unicode_ci

/*
3 rows from SalesData table:
id	salesperson_id	product_id	customer_id	region_id	amou

'The SalesData table has the following columns: id, salesperson_id, product_id, customer_id, region_id, amount, total, and sales_date. It contains information about sales transactions.'

In [34]:
agent_executor.run("列出华东大区域6月份的销售总额")



> Entering new  chain...
Action: sql_db_list_tables
Action Input: ""
Observation: Customer, Product, Region, SalesData, Salesperson
Thought:The relevant tables for this question are SalesData and Region. I should query the schema of these tables to see what columns I can use to calculate the sales total.
Action: sql_db_schema
Action Input: "SalesData, Region"
Observation: 
CREATE TABLE `Region` (
	id INTEGER NOT NULL AUTO_INCREMENT, 
	region_name VARCHAR(255) COLLATE utf8mb4_unicode_ci, 
	PRIMARY KEY (id)
)ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_unicode_ci

/*
3 rows from Region table:
id	region_name
1	东北
2	华北
3	华东
*/


CREATE TABLE `SalesData` (
	id INTEGER NOT NULL AUTO_INCREMENT, 
	salesperson_id INTEGER, 
	product_id INTEGER, 
	customer_id INTEGER, 
	region_id INTEGER, 
	amount INTEGER, 
	total DECIMAL(10, 2), 
	sales_date DATE, 
	PRIMARY KEY (id), 
	CONSTRAINT salesdata_ibfk_1 FOREIGN KEY(salesperson_id) REFERENCES `Salesperson` (id), 
	CONSTRAINT salesdata_ibfk_2 

'79367.50'

In [57]:
agent_executor.run("请统计华东大区所有销售人员的数量，并列出他们的姓名，年龄和电话, 请用中文回答")



> Entering new  chain...
Action: sql_db_list_tables
Action Input: ""
Observation: Customer, Product, Region, SalesData, Salesperson
Thought:

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised ServiceUnavailableError: The server is overloaded or not ready yet..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer')).


I can query the Salesperson table to get the information about the salespeople in the East region.
Action: sql_db_schema
Action Input: "Salesperson"
Observation: 
CREATE TABLE `Salesperson` (
	id INTEGER NOT NULL AUTO_INCREMENT, 
	name VARCHAR(255) COLLATE utf8mb4_unicode_ci, 
	gender CHAR(1) COLLATE utf8mb4_unicode_ci, 
	birthday DATE, 
	address VARCHAR(255) COLLATE utf8mb4_unicode_ci, 
	id_card VARCHAR(255) COLLATE utf8mb4_unicode_ci, 
	phone VARCHAR(255) COLLATE utf8mb4_unicode_ci, 
	performance_score DECIMAL(5, 2), 
	performance_bonus DECIMAL(10, 2), 
	position VARCHAR(255) COLLATE utf8mb4_unicode_ci, 
	join_date DATE, 
	region_id INTEGER, 
	PRIMARY KEY (id), 
	CONSTRAINT salesperson_ibfk_1 FOREIGN KEY(region_id) REFERENCES `Region` (id)
)ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_unicode_ci

/*
3 rows from Salesperson table:
id	name	gender	birthday	address	id_card	phone	performance_score	performance_bonus	position	join_date	region_id
1085	叶丽华	M	1993-02-10	湖南省合肥县兴山罗街d座 7

'华东大区所有销售人员的数量为3人，他们的姓名、年龄和电话分别是叶丽华（30岁，14570343311）、焦林（50岁，15386680111）和林洋（43岁，13261605440）。'